In [ ]:
import pandas as pd
import numpy as np
import yfinance as yf

In [260]:
TRANSACTIONS_STANDARD_COLS = ['date', 'account', 'symbol', 'action', 'quantity', 'price', 'amount']
ACTIONS_STANDARD = ['BUY', 'SELL', 'DIVIDEND', 'SPLIT', 'ACH', 'INTEREST EARNED', 'INTEREST PAID', 'FEE', 'TRANSFER']
TAX = {'ST': 0.4, 'LT': 0.2}

# Adjustment to stock price (Adjusted Closing Price): https://help.stockcharts.com/learning-more/policies-and-limitations/data-policies


In [261]:
# Marcus Invest transactions
marcus = pd.read_csv('marc8202_20250215.csv')
marcus['account'] = 'marc8202'
marcus_actions = {
    'A': 'ACH',
    'B': 'BUY',
    'C': 'CAP GAIN',
    'D': 'DIVIDEND',
    'F': 'FEE',
    'S': 'SELL',
    'T': 'TRANSFER'
}
marcus['action'] = marcus['Transaction'].map(marcus_actions)
marcus['date'] = pd.to_datetime(marcus['Date'])
marcus['symbol'] = marcus['Desc']
marcus['quantity'] = np.abs(marcus['Quantity'])
marcus['Credit'] = marcus['Credit'].str.replace('$','').str.replace(',','').astype(float)
marcus['Debit'] = marcus['Debit'].str.replace('$','').str.replace(',','').astype(float)
marcus['amount'] = marcus['Credit'] - marcus['Debit']
marcus['price'] = marcus['Price'].str.replace('$','').str.replace(',','').astype(float)

marcus = marcus[TRANSACTIONS_STANDARD_COLS]

# B	BOUGHT
# S	SOLD
# D	DIVIDEND
# A	ACH
# F	FEE
# C	CAP GAIN
# T	TRANSFER

In [262]:
# Schwab transactions
schwab = pd.read_csv('schwab576_20250215.csv')
schwab['account'] = 'schwab576'

# ['Non-Qualified Div', 'Cash Dividend', 'Margin Interest',
#        'Qualified Dividend', 'MoneyLink Transfer', 'Credit Interest',
#        'Buy', 'Journal', 'Sell', 'Security Transfer']

marcus_actions = {
    'Non-Qualified Div': 'DIVIDEND',
    'Cash Dividend': 'DIVIDEND', 
    'Margin Interest': 'INTEREST PAID',
    'Qualified Dividend': 'DIVIDEND',
    'MoneyLink Transfer': 'ACH',
    'Credit Interest': 'INTEREST EARNED',
    'Buy': 'BUY',
    'Journal': 'TRANSFER',
    'Sell': 'SELL',
    'Security Transfer': 'TRANSFER',
}
# schwab.columns = ['date', 'action', 'symbol', 'description', 'quantity', 'price', 'fees', 'amount']
schwab['date'] = pd.to_datetime(schwab['Date'].str[-10:], format='%m/%d/%Y')
schwab['action'] = schwab['Action'].map(marcus_actions)
schwab['symbol'] = schwab['Symbol']
schwab['quantity'] = schwab['Quantity']
schwab['price'] = schwab['Price'].str.replace('$','').str.replace(',','').astype(float)
schwab['amount'] = schwab['Amount'].str.replace('$','').str.replace(',','').astype(float)
schwab = schwab[TRANSACTIONS_STANDARD_COLS]

In [263]:
txns = pd.concat([marcus, schwab])
txns['adj_quantity'] = np.where(txns['action'] == 'SELL', -txns['quantity'], txns['quantity'])

In [264]:
stock_txns = txns[txns.symbol.notna()&(txns.symbol!='Cash')&(txns.action.isin(['BUY', 'SELL']))]
stock_txns['lot_number'] = stock_txns.groupby(['symbol', 'action'])['date'].rank(method='first')
stock_txns = stock_txns.sort_values(['symbol', 'date'])
stock_txns['cum_lot_quantity'] = stock_txns.groupby(['symbol'])['adj_quantity'].cumsum()
stock_txns = stock_txns.merge(stock_txns[stock_txns.action=='SELL'].groupby('symbol')['adj_quantity'].sum().rename('cum_sell_quantity'), how='left', on='symbol')
stock_txns['cum_sell_quantity'] = stock_txns['cum_sell_quantity'].fillna(0)

C:\Users\sande\AppData\Local\Temp\ipykernel_3928\118986780.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stock_txns['lot_number'] = stock_txns.groupby(['symbol', 'action'])['date'].rank(method='first')


In [265]:
# Current portfolio Snapshot

# Identifying sold lots
stock_txns['sold_status'] = np.where((stock_txns['cum_lot_quantity'] + stock_txns['cum_sell_quantity']) <= 0.0, 'SOLD', 'HOLDING')
stock_txns['cum_net_holding'] = stock_txns['cum_lot_quantity'] + stock_txns['cum_sell_quantity']
stock_holding = stock_txns[(stock_txns['sold_status']=='HOLDING')&(stock_txns['action']=='BUY')]
stock_holding = stock_holding.sort_values(['symbol', 'date'])
stock_holding['net_holding'] = stock_holding.groupby('symbol')['cum_net_holding'].transform('diff').combine_first(stock_holding['cum_net_holding'])
stock_holding['holding_period_days'] = (pd.Timestamp.now() - stock_holding['date']).dt.days
stock_holding = stock_holding[stock_holding['net_holding']>0.1]
stock_holding['LT_quantity'] = np.where(stock_holding['holding_period_days'] > 365, stock_holding['net_holding'], 0)
stock_holding['invested'] = stock_holding['net_holding'] * stock_holding['price']
stock_holding['holding_period_days_invested'] = stock_holding['holding_period_days'] * stock_holding['invested']

snapshot = stock_holding[['symbol', 'net_holding', 'invested', 'holding_period_days_invested', 'LT_quantity']].groupby('symbol').sum()
snapshot['cost_basis'] = snapshot['invested'] / snapshot['net_holding']
snapshot['avg_holding_days'] = snapshot['holding_period_days_invested'] / snapshot['invested']

def get_holding_dict(df: pd.DataFrame) -> pd.Series:
    holding_dict = dict()
    for symbol, data in df.groupby('symbol'):
        holding_dict[symbol] = dict(zip(data.date.astype(str), data.net_holding))
    return pd.Series(holding_dict)

snapshot = snapshot[['net_holding', 'invested', 'cost_basis', 'avg_holding_days', 'LT_quantity']].join(get_holding_dict(stock_holding).rename('lots'))


In [ ]:
# Get current stock price
# Closing price adjustments: https://help.stockcharts.com/learning-more/policies-and-limitations/data-policies (They keep changing everyday)
# stock_prices = yf.download(list(snapshot.index))['Adj Close']
stock_prices
latest_prices = stock_prices.iloc[-1]
snapshot['current_price'] = latest_prices
snapshot['current_value'] = snapshot['current_price'] * snapshot['net_holding']

[*********************100%***********************]  29 of 29 completed


,net_holding,invested,cost_basis,avg_holding_days,LT_quantity,lots,current_price,current_value
symbol,,,,,,,,
ADBE,10.00000,4750.000000,475.000000,65.000000,0.00000,{'2024-12-13': 10.0},460.160004,4601.600037
AMD,50.00000,6399.790000,127.995800,59.531530,0.00000,"{'2024-12-13': 30.0, '2024-12-27': 20.0}",113.099998,5654.999924
AMZN,16.00000,2889.600000,180.600000,201.854790,0.00000,"{'2024-07-17': 1.0, '2024-07-30': 10.0}",228.679993,3658.879883
AVGO,10.00000,2185.000000,218.500000,59.000000,0.00000,{'2024-12-19': 10.0},233.039993,2330.399933
BA,10.00000,1680.000000,168.000000,62.000000,0.00000,{'2024-12-16': 10.0},184.419998,1844.199982
CART,30.00000,1020.000000,34.000000,279.000000,0.00000,{'2024-05-13': 30.0},50.970001,1529.100037
CFLT,77.00000,2321.680000,30.151688,268.290703,0.00000,"{'2024-05-20': 50.0, '2024-05-28': 10.0, '2024...",34.849998,2683.449883
CI,25.00000,7000.000000,280.000000,65.000000,0.00000,{'2024-12-13': 25.0},292.320007,7308.000183
CMCSA,10.00000,400.000000,40.000000,65.000000,0.00000,{'2024-12-13': 10.0},35.389999,353.899994
